In [3]:
from twython import Twython
from twython import TwythonError

import pandas as pd

In [ ]:
# we look at the user account - who is @cchqpress?

twtool.show_user(screen_name = 'CCHQPress')

In [ ]:
# now, their timeline

tweets = twtool.get_user_timeline(screen_name = 'CCHQPress', tweet_mode = "extended", count = 5000)


In [ ]:
# we look at a tweet

tweets[117]

## Collecting the tweets

In [ ]:
# this will get us to a 3k twitter limit

all_tweets = []

# we collect on the first page of results - here 'user_timeline' is a simple request

try:
    user_timeline = twtool.get_user_timeline(screen_name = 'CCHQPress', tweet_mode = "extended", count = 200)

except TwythonError as e:
    print (e)

# just so we can see it running
print (len(user_timeline))

for tweet in user_timeline:
    tweets = []
    tweets.append(tweet['id'])
    clean_time = tweet['created_at']
    tm = clean_time[:-11]
    tweets.append(tm)
    tweets.append(tweet['full_text'])
    all_tweets.append(tweets)

    
# this runs while the more specific 'user_timeline' brings back a batch of results that is not 0    
while len(user_timeline) != 0: 
    
    
    # the max_id parameter is what allows us to continue
    try:
        user_timeline = twtool.get_user_timeline(screen_name='CCHQPress', tweet_mode = "extended", count = 200, max_id = user_timeline [len(user_timeline)-1]['id']-1)
    
    except TwythonError as e:
        print (e)
        
    print (len(user_timeline))
    
    # loop through the tweets, appending ID, TIME (cleaned from unix time) and FULL TEXT
    for tweet in user_timeline:
        tweets = []
        tweets.append(tweet['id'])
        clean_time = tweet['created_at']
        tm = clean_time[:-11]
        tweets.append(tm)
        tweets.append(tweet['full_text'])
        
        # append this list we've just made with id, time and content...to the main list 'all tweets'
        all_tweets.append(tweets)


In [ ]:
# how many were collected at 23h on Nov 24?

len(all_tweets)

## Make and clean a df

In [ ]:
# declare column names in a list, then build the df

col_names = ["tweet_id", "time", "content"]

df_recent = pd.DataFrame(all_tweets, columns = col_names)

In [ ]:
# flip the df so it moves in chronological direction

df = df_recent.reindex(index = df_recent.index[::-1])

In [ ]:
# reset the index to run from 0

df = df.reset_index()

In [ ]:
# drop an extra index column that's been created

df = df.drop(df.columns[0], axis=1)

In [ ]:
# identify the tweets we want from the timings 

df[2603:2817]

In [ ]:
# nake a new df from the selection

debate = df[2603:2817]

In [ ]:
# how many tweets?

len(debate)

In [ ]:
# how many of these tweets are retweets?

len(debate[debate['content'].str.contains('RT @')])

## Filtering out the retweets

In [ ]:
# we build a simple filter for text in a column:
# df[df['column'].str.contains('some text')] 
# and flip it using ~ to get only those rows that DON'T contain it

originals = debate[~debate['content'].str.contains('RT @')]

In [ ]:
# force the df to display across the page - it works but generates \n

pd.set_option('display.max_colwidth', -1)

originals

In [ ]:
# lambda functions that replace the line breaks with spaces

rep_doubles = lambda x: x.replace('\n\n', ' ')
rep_singles = lambda x:x.replace('\n', ' ')
rep_and = lambda x:x.replace('&amp;', '&')

In [ ]:
# Use .apply to map the functions across the entire column

originals["content"] = originals["content"].apply(rep_doubles)

In [ ]:
originals["content"] = originals["content"].apply(rep_singles)

In [ ]:
originals["content"] = originals["content"].apply(rep_and)

In [ ]:
originals

## Reading the tweets

In [ ]:
# with Row here and CTRL + ENTER we can click through the quotes

row = 0

In [ ]:
#print (originals.iat[row,0])
#print (originals.iat[row,1])
print (originals.iat[row,2])

row = row + 1